In [291]:
from sympy import *

In [292]:
def sympy_to_text(expr):
    # return str(expr).replace('*', ' * ').replace('+', ' + ').replace('/', ' / ')
    return str(expr)


class CodeGenerator:
    def __init__(self):
        self.tabs = 0
        self.text = ""

    def tab(self):
        self.tabs += 1

    def untab(self):
        self.tabs -= 1

    def __iadd__(self, text):
        self.text += "\t" * self.tabs + text + "\n"
        return self
        
    def return_vec(self, matrix):
        self += "return Vec3("
        self.tab()
        for i in range(3):
            text = sympy_to_text(matrix[i])
            if i != 2:
                text += ","
            self += text
        self.untab()
        self += ")"

    def function_begin(self, return_type, type_name, function_name):
        self += f"{return_type} {type_name}::{function_name}(f32 u, f32 v) {{"
        self.tab()

    def function_end(self):
        self.untab()
        self += "}"

    def vector_function(self, type_name, function_name, return_value):
        self.function_begin("Vec3", type_name, function_name)
        self.return_vec(return_value)
        self.function_end()

    def newline(self):
        self.text += "\n"


In [293]:
u, v = symbols('u, v')

def surface_data(x, y, z, print_formulas, generate_code, surface_name):
    surface = Matrix([[x], [y], [z]])

    coordinates = [u, v]
    jacobian = simplify(surface.jacobian(Matrix(coordinates)))
    metric = simplify(jacobian.T * jacobian)

    metricInv = simplify(metric.inv())

    coorindate_count = 2
    cristoffel = [ 
        [ 
            [Integer(0), Integer(0)],
            [Integer(0), Integer(0)] 
        ],
        [ 
            [Integer(0), Integer(0)],
            [Integer(0), Integer(0)] 
        ] 
    ]

    # https://math.stackexchange.com/questions/1720424/how-to-best-calculate-christoffel-symbols-of-metric-du2-g2u-dv2
    for i in range(coorindate_count):
        for j in range(coorindate_count):
            x_j = coordinates[j]
            for k in range(coorindate_count):
                x_k = coordinates[k]
                for m in range(coorindate_count):
                    x_m = coordinates[m]
                    cristoffel[i][j][k] += metricInv[i, m] * surface.diff(x_j, x_k).dot(surface.diff(x_m))

    for i in range(coorindate_count):
        cristoffel[i] = trigsimp(simplify(Matrix(cristoffel[i])))

    x_u = jacobian.col(0)
    x_v = jacobian.col(1)

    # https://stackoverflow.com/questions/39691325/display-a-matrix-with-putting-a-common-factor-in-sympy
    normal = simplify(x_u.cross(x_v))
    normal = expand_trig(normal)
    normal /= gcd(tuple(normal))

    # display(cse(cristoffel[0]))

    if print_formulas:
        print('Cristoffel symbols')
        display(cristoffel[0])
        display(cristoffel[1])

        print('x_u')
        display(x_u)

        print('x_v')
        display(x_v)

        print('normal')
        display(normal)
    
    if generate_code:
        from sympy.utilities.codegen import codegen, InputArgument
        from sympy.codegen.ast import Assignment
        out = MatrixSymbol("out",2,2)
        # [(c_name, c_code), (h_name, c_header)] = codegen((
        #     'test',
        #     Equality(out, cristoffel[0],evaluate=False) ), 
        #     'C', 
        #     argument_sequence=(u, v, out)
        # )
        print(ccode(Assignment(out, cristoffel[0])))
        # print(c_code)
        # c = CodeGenerator()
        # vector_functions = [
        #     ('position', surface),
        #     ('tangentU', x_u),
        #     ('tangentV', x_v),
        #     ('normal', normal)
        # ]
        # for name, vector in vector_functions:
        #     c.vector_function(surface_name, name, vector)
        #     c.newline()

        # c.function_begin("Mat2", surface_name, "christoffelSymbols")
        # c += "return {"
        # c.tab()
            
        # for i in range(2):
        #     c += f".{["x", "y"][i]} = Mat2("
        #     c.tab()
        #     c += f"Vec2({sympy_to_text(cristoffel[i][0, 0])}, {sympy_to_text(cristoffel[i][1, 0])}),"
        #     c += f"Vec2({sympy_to_text(cristoffel[i][0, 1])}, {sympy_to_text(cristoffel[i][1, 1])})"
        #     c.untab()
        #     c += ")"

        # c.untab()
        # c += "}"
        # c.function_end()
        # print(c.text)


In [294]:
# r = symbols('r')
# x = r * sin(u) * cos(v)
# y = r * sin(u) * sin(v)
# z = r * cos(u)
# surface_data(x, y, z, False, True, "Sphere")

In [297]:
# surface_data(
#     (1 + Rational(1, 2) * v * cos(Rational(1, 2) * u)) * cos(u),
#     (1 + Rational(1, 2) * v * cos(Rational(1, 2) * u)) * sin(u),
#     Rational(1, 2) * v * sin(Rational(1, 2) * u),
#     False, True, "MobiusStrip"
# )

out[0] = -v*(v*sin(u) + 4*sin((1.0/2.0)*u))/(2*pow(v, 2)*cos(u) + 3*pow(v, 2) + 16*v*cos((1.0/2.0)*u) + 16);
out[1] = (4*v*pow(cos((1.0/2.0)*u), 2) + v + 8*cos((1.0/2.0)*u))/(4*pow(v, 2)*pow(cos((1.0/2.0)*u), 2) + pow(v, 2) + 16*v*cos((1.0/2.0)*u) + 16);
out[2] = (4*v*pow(cos((1.0/2.0)*u), 2) + v + 8*cos((1.0/2.0)*u))/(4*pow(v, 2)*pow(cos((1.0/2.0)*u), 2) + pow(v, 2) + 16*v*cos((1.0/2.0)*u) + 16);
out[3] = 0;
